# The Goalie Bot

###### Alex Reichenbach, Darcy Meyer, Gherardo Morona, Jocelyn Shen, Hugh Jones, Neil Shroff

<img src="Picture1.png">

In [3]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy import stats
import math
#import rospy
#from geometry_msgs.msg import Twist

In [18]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/zAfiCy9muuA" frameborder="0" allowfullscreen></iframe>')

# What it does

The Goalie bot predicts the path of a ball, in order to intercept it

# How it works

1. The robot processes the image of the ball using 
1. It then converts the images to coordinates in a 2D plane
1. Then, it performs a linear regression on these coordinates to predict the line the ball will follow
1. The Goalie Bot calculates the shortest time to intercept the path, and moves to block the ball

# 1. Image Processing

#### *Sensors and Communication with the Turtlebot*

In [4]:
from PIL import Image, ImageDraw

In [5]:
img = Image.open('20161104_172958.jpg')
if type(img) == None:
    print("ERROR! IMAGE NOT FOUND")

In [6]:
def findBall(r, g, b, tol, image):
    rLower = r-tol
    rUpper = r+tol
    gLower = g-tol
    gUpper = g+tol
    bLower = b-tol
    bUpper = b+tol
    xBall = []
    yBall = []
    width, height = image.size
    for yCoord in range(0, height, 3):
        for xCoord in range(0, width, 3):
            r, g, b = image.getpixel((xCoord, yCoord))
            #Please note imread is in BGR!
            if g > b + tol and g > r + tol:
                    xBall.append(xCoord)
                    yBall.append(yCoord)
    return (sum(xBall)/len(xBall), sum(yBall)/len(yBall))
                    

In [7]:
coords = findBall(100, 100, 90, 10, img)
draw = ImageDraw.Draw(img)
draw.ellipse((coords[0]-20, coords[1]-20, coords[0]+20, coords[1]+20), fill = 'blue', outline ='blue')
img.show()

In [8]:
print(findBall(110, 149, 90, 10, img))

(3544.1650000447617, 1476.6039049980752)


In [9]:
CAMERA_ANGLE = 57*math.pi/180 #the camera horizontal angle is 57 degrees
NUM_PIXLES = img.size[0]

def find_pos(depth, xpos):
    r = depth
    theta = (xpos-NUM_PIXLES/2)/(NUM_PIXLES/2)*CAMERA_ANGLE/2
    x = r*cos(theta)
    y = r*sin(theta)
    return (x, y)

###### Finding the ball with the RGB Camera

In [10]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/M_ruYUJyrnw" frameborder="0" allowfullscreen></iframe>')

# 2. Predicting the Path of the Ball

###### *Calculating the shortest distance to the path*

#### *Sample Dataset*

In [11]:
points_x = [1, 2, 3.1, 4.2, 5.1, 4.9]
points_y = [1.1,2,3,4,5,4.8]
points = {"x": points_x, "y": points_y}
df = pd.DataFrame(points)
linreg = LinearRegression()
X = df.x
X = X.reshape(-1,1)
y = df.y
linreg.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

###### Plot the path of the ball and the coordinates of the ball that are read from the robot

$$y = m\cdot x + b$$

In [12]:
m = linreg.coef_[0]
b = linreg.intercept_ 
x = np.linspace(0,max(df["x"]), 10)
abline_values = [m * i + b for i in x]
df.plot("x","y", kind = "scatter")
plt.plot(x, abline_values, 'b')
plt.show()

# 3. Intercepting the Ball

###### Find the equation of the line that the robot must follow, perpendicular to the line the ball follows

$$y_b = m_b\cdot x$$

In [13]:
m_bot = -1/m
b_bot = 0
x = np.linspace(0,max(df["x"]), 10)
abline_values = [m_bot * i + b_bot for i in x]
df.plot("x","y", kind = "scatter")
plt.plot(x, abline_values, 'b')
plt.show()

###### Calculate the point of intersection between the path the ball follows, and the path the robot follows

$$m\cdot x + b = m_b\cdot x$$
$$x = \frac{b}{m_b - m}$$
$$y = m_b\cdot\frac{b}{m_b - m}$$

In [14]:
x1 = b/(-m + m_bot)
y1 = m_bot*b/(-m+m_bot)

###### Calculate the distance and angle between the point of intersection and the location of the robot (the origin)

$$d_b = \sqrt {\left( {x} \right)^2 + \left( {y} \right)^2 }$$

In [15]:
k = 1
distance = ((x1*k)**2 + (y1*k)**2)**0.5
VELOCITY = 0.5
time = distance/VELOCITY #time in seconds
angle = math.atan(y1/x1)

In [16]:
#inspired by code from https://github.com/markwsilliman/turtlebot/blob/master/goforward.py
class MoveForward():
    def __init__(self):
        rospy.init_node('GoForward', anonymous=False)
        rospy.on_shutdown(self.shutdown)
        
        self.cmd_vel = rospy.Publisher('cmd_vel_mux/input/navi', Twist, queue_size=10)
        
        # Move at a rate of 10 HZ
        r = rospy.Rate(10);
        
        move_cmd = Twist()
        # move at VELOCITY m/s
        move_cmd.linear.x = VELOCITY
        # turn at angle radians/s
        move_cmd.angular.z = angle

        while not rospy.is_shutdown():
            # publish the velocity
            self.cmd_vel.publish(move_cmd)
            # publish again after 10 HZ (0.1 s)
            r.sleep()
        
    def shutdown(self):
        # stop turtlebot
        rospy.loginfo("Stop TurtleBot")
        # publish default Twist to stop TurtleBot
        self.cmd_vel.publish(Twist())
        # confirm TurtleBot receives command before shutting down
        rospy.sleep(1)

In [17]:
start = rospy.Time.now()
while(rospy.Time.now() - start < rospy.Duration(time)):
    try:
        GoForward()
    except:
        rospy.loginfo("Uh oh! The MoveForward node has been terminated")

NameError: name 'rospy' is not defined

# Integrating with the robot

In [ ]:
from __future__ import print_function
import roslib
import sys
import rospy
import cv2
from std_msgs.msg import String
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError
from geometry_msgs.msg import Twist
from sensor_msgs.msg import PointCloud2
import sensor_msgs.point_cloud2 as pc2
import struct
import math
import numpy as np
from sklearn.linear_model import LinearRegression 

class GoalieBot:
	limiter = 0
	maxCount = 4
	skipPixels = 9
	recentDepthImage = None
	CAMERA_ANGLE = 57*math.pi/180 #the camera horizontal angle is 57 degrees
	NUM_PIXLES = 640
	cameraAngle = 21
	distanceFromGoal = 1.5
	ball = None
	ballPositions = []
	positions = []
	shouldContinue = True
	class Ball:
		SPEC = 40
		LOW_SPEC = 150
		XImagePos = 0.0
		YImagePos = 0.0
		distance = 0.0

	def __init__(self):
		self.bridge = CvBridge()
		# look at the rgb image		
		self.image_sub = rospy.Subscriber("/camera/rgb/image_raw",Image, self.ballTrack, queue_size=5)
		
		# give commands to the wheels		
		self.movement = rospy.Publisher("/mobile_base/commands/velocity", Twist, queue_size=5)

		# look at the depth image 
		self.image_depth = rospy.Subscriber("/camera/depth/image_raw", Image, self.depth_test, queue_size=5)

	def depth_test(self,data):
		# storing the depth image
		data.encoding = "mono16"
		self.recentDepthImage = self.bridge.imgmsg_to_cv2(data, "passthrough")
		# open the depth image
		if self.ball == None:
			return
		cv2.circle(self.recentDepthImage, (int(self.ball.YImagePos), int(self.ball.XImagePos)), 25, 100000, thickness=10)
		#cv2.imshow("", self.recentDepthImage)
		#cv2.waitKey(3)

	def getPosInDepthImage(self, x, y):
		assert self.recentDepthImage != None
		image = self.recentDepthImage
		x = int(x*image.shape[0]+0.5)
		y = int(y*image.shape[1]+0.5)
		rect_size = 10
		min_depth = image[0][0]
		for i in range(x-rect_size, x+rect_size):
			if i > 0 and i < image.shape[0]:
				for j in range(y-rect_size, y+rect_size):
					if j > 0 and j < image.shape[1]:
						if image[x][y] != 0:
							min_depth = min(image[x][y], min_depth)
		return min_depth

	def ballTrack(self,data):
		if self.shouldContinue == False or self.recentDepthImage == None:
			return
		if self.limiter < self.maxCount:
			self.limiter+=1
			return
		self.limiter = 0
		try:
			img = self.bridge.imgmsg_to_cv2(data, "bgr8")
		except CvBridgeError as e:
			print(e)
		self.findBall(img)
		twister = Twist()
		twister.angular.z = 0
		if self.ball != None:
			#if ball.XImagePos < img.shape[0]/2:
			#	twister.angular.z = -1
			#else:
			#	twister.angular.z = 1
			cv2.ellipse(img, (self.ball.YImagePos, self.ball.XImagePos), (20, 20), 0, 0, 180, (255,0,0), 2)
			#cv2.imshow("Image window", img)
			ballCoord = self.polarToCartesian(self.ball.distance, self.ball.YImagePos, img.shape[1])
			self.positions.append(ballCoord)
			if ballCoord[1] < self.distanceFromGoal:
				self.interceptBall()
			print("Found ball! Distance is", ballCoord)
			#cv2.waitKey(3)
		else:
			print("Can't find ball!")
		self.movement.publish(twister)

	def interceptBall(self):
		linreg = LinearRegression()
		X = [x[0] for x in self.positions]
		y = [x[1] for x in self.positions]
		linreg.fit([[Y] for Y in y],[[x] for x in X] )
		#Remove Outliers

		errors = []
		for i in range(len(X)):
			predicted = linreg.predict(X[i])
			error = abs(predicted - y[i])
			errors.append(errors)
		for i in range(int(len(X)/20)):
			max_error = errors.index(max(errors))
			X.pop(max_error)
			y.pop(max_error)
		linreg.fit([[Y] for Y in y], [[x] for x in X])

		overshootConstant = 1.8

		XBallPredicted = float(linreg.predict(0)[0][0])*overshootConstant
	

		# Move at a rate of 10 HZ
		r = rospy.Rate(10);
		print("Predicted: ", XBallPredicted)
		speed = .6# in m/s: max of .65
		updateRate = 40.0
		r = rospy.Rate(updateRate);
		print("GOING FOR IT!!!!")
		for i in range(int(abs(XBallPredicted)*updateRate*(1.0/speed))):
			move_cmd = Twist()
			k=1
			# turn at angle radians/s
			if XBallPredicted < 0:
				k = -1
			move_cmd.linear.x = k*speed
			# publish the velocity
			self.movement.publish(move_cmd)
			# publish again after 10 HZ (0.1 s)
			r.sleep()

		self.shouldContinue = False


	def findBall(self, image):
		# find the ball in the rgb image, set ball's x-pixel, y-pixel, distance from camera
		width, height, channel = image.shape
		minDepth = 100000000
		sumXBalls = 0
		sumYBalls = 0
		countX = 0
		countY = 0
		for yCoord in range(0, height, self.skipPixels):
			for xCoord in range(0, width, self.skipPixels):
				b, g, r = image[xCoord,yCoord]
				#Please note imread is in BGR!
				if g > b + self.Ball.SPEC and g > r + self.Ball.SPEC and g > self.Ball.LOW_SPEC:
					sumXBalls += xCoord
					sumYBalls += yCoord
					countX += 1
					countY += 1
		if countX == 0:
			return None
		if self.ball == None:
			self.ball = self.Ball()
		self.ball.XImagePos = sumXBalls/countX
		self.ball.YImagePos = sumYBalls/countY
		k = 2.4865738729
		cameraHeight = .2
		pixelHeight = self.ball.YImagePos
		y = self.ball.XImagePos - width/2
		phi = math.atan(2*y*math.tan(57/2*math.pi/180)/width);
		d = math.tan(math.pi/2 - phi - self.cameraAngle*math.pi/180)*cameraHeight*k
		self.ball.distance = d		

	def polarToCartesian(self, depth, xpos, width_in_pix):
		r = depth
		#theta = float(-1)*(xpos-self.NUM_PIXLES/2)/(self.NUM_PIXLES/2)*self.CAMERA_ANGLE/2 + 90
		start_angle = 90 - 57/2
		theta = float(width_in_pix - xpos)/float(width_in_pix) * 57 + start_angle
		#x = self.ball.YImagePos - width_in_pix/2
		#phi = math.atan(2*x*math.tan(57/2*math.pi/180)/	width_in_pix)
		#print(width_in_pix, self.ball.YImagePos)
		#theta = math.tan(math.pi/2 - phi)
		y_coord = r*math.sin(theta*math.pi/180)
		x_coord = r*math.cos(theta*math.pi/180)
		return (x_coord, y_coord)

def main():
	rospy.init_node('goalieBot', anonymous=True)
	#ic = image_converter()
	goalieBot = GoalieBot()
	try:
		rospy.spin()
	except KeyboardInterrupt:
		print("Shutting down")
	cv2.destroyAllWindows()

main()

# Results

### Sample data

Found ball! Distance is (0.13993161801956358, 1.6512788514166552)

Found ball! Distance is (0.0010888493947457053, 1.8148790504430792)

Found ball! Distance is (-0.1570943625997164, 1.9538323337502883)

Found ball! Distance is (-0.037961547649821806, 1.793576716255294)

Found ball! Distance is (0.014348498671807186, 1.7138016068301745)

Found ball! Distance is (-0.2932684091690599, 1.9380745773452985)

Found ball! Distance is (-0.3593614163994403, 1.926914320765741)

Found ball! Distance is (-0.36120709382093025, 1.9882106610460286)

Found ball! Distance is (-0.3609753958873394, 1.9027279313177135)

Found ball! Distance is (-0.18436324326141793, 1.774108941201135)

Found ball! Distance is (-0.18092457284491842, 1.7952900822366742)

Found ball! Distance is (-0.1872045044284429, 1.723246748287013)

Found ball! Distance is (-0.0702651367752794, 1.5253276590989073)

Found ball! Distance is (0.07304176252044554, 1.2226740500630218)

Found ball! Distance is (0.1686843762348317, 0.8144341936266234)

### Test run 1

In [ ]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/raBsW7Jb58s" frameborder="0" allowfullscreen></iframe>')